In [90]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import mean_absolute_error
# from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeRegressor
# from sklearn.svm import SVR
# from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor, ExtraTreesRegressor
# import xgboost as xgb
import pickle
import plotly.graph_objects as go

In [91]:
monthly_data=pd.read_csv("SKU_Monthly_training_data.csv",usecols=["Name","Month","Year","Count"])
weekly_data=pd.read_csv("SKU_Weekly_training_data.csv",usecols=["Name","Month","Year","Count","wom"])
prices=pd.read_csv("proc_prices_table_2018.csv",usecols=["Name","Avg Unit Price"],encoding='utf-8-sig')
monthly_operations=pd.read_csv("Monthly_Training_Data.csv",usecols=["Service Name","Month","Year","Count"],encoding='utf-8-sig')
op_SKU=pd.read_csv("operation_SKUvalues.csv",encoding='utf-8-sig')
op_SKU.reset_index(drop=True,inplace=True)
inv_df=pd.read_csv("Inventory_Cost.csv",usecols=["Type","Month","Year","Count","Avg Unit Price"])

# print(weekly_data.shape,monthly_data.shape)
op_SKU.head()

,Name_AR,Average,Avg Unit Price
0,ترقيع للملتحمه,8,382.701795
1,D.C.R. - العينين,25,544.536480
2,D.C.R. - عين واحدة,31,737.147449
3,LASIK,64,1672.320222
4,LASIK*,9,280.886588


In [92]:
inv_df.rename(columns={"Avg Unit Price":"Inventory_price","Count":"Inventory_count"},inplace=True)
inv_df.head()

,Type,Month,Year,Inventory_count,Inventory_price
0,أوبسيت صغير,1,2018,3100,9.333333
1,أوبسيت صغير,2,2018,3620,9.333333
2,أوبسيت صغير,3,2018,3220,9.333333
3,أوبسيت صغير,4,2018,3520,9.333333
4,أوبسيت صغير,5,2018,4020,9.333333


In [93]:
prices["Avg Unit Price"].max()

37433.33333333333

In [94]:
# # monthly_data=monthly_data.sort_values(by=["Count"],ascending=False).iloc[0:100]
# monthly_data=monthly_data[(monthly_data['Year'] == 2018) & (monthly_data['Month'] == 12)]
# monthly_data=monthly_data.sort_values(by=["Count"],ascending=False).iloc[0:20]
# monthly_data

In [95]:
with open('operations_names_en_to_ar.pickle', 'rb') as f:
    operations_names_en_to_ar = pickle.load(f)

In [96]:
# Service_Names_to_index = {name : idx for idx, name in enumerate(weekly_data['Service Name'].unique())}
# Service_Index_to_name= {idx : name for idx, name in enumerate(weekly_data['Service Name'].unique())}
# monthly_Service_Index_to_name= {idx : name for idx, name in enumerate(weekly_data['Service Name'].unique())}

In [97]:
def encode_monthly_names(df):
    Service_Names_to_index = {name : idx for idx, name in enumerate(monthly_data['Name'].unique())}
    df['Name']=df['Name'].map(Service_Names_to_index)
    return df

In [98]:
def decode_monthly_names(df):
    Service_Index_to_name= {idx : name for idx, name in enumerate(monthly_data['Name'].unique())}
    df['Name']=df['Name'].map(Service_Index_to_name)
    return df

In [99]:
def encode_weekly_names(df):
    Service_Names_to_index = {name : idx for idx, name in enumerate(weekly_data['Name'].unique())}
    df['Name']=df['Name'].map(Service_Names_to_index)
    return df

In [100]:
def decode_weekly_names(df):
    Service_Index_to_name= {idx : name for idx, name in enumerate(weekly_data['Name'].unique())}
    df['Name']=df['Name'].map(Service_Index_to_name)
    return df

In [101]:
weekly_processed_data = weekly_data.copy()
weekly_processed_data=encode_weekly_names(weekly_processed_data)
monthly_processed_data= monthly_data.copy()
monthly_processed_data=encode_monthly_names(monthly_processed_data)
monthly_processed_data.head()
# monthly_processed_data=encode_monthly_names(monthly_data)

,Name,Count,Month,Year
0,0,9,1,2016
1,1,8,1,2016
2,2,9,1,2016
3,3,8,1,2016
4,4,723,1,2016


In [102]:
# X=data.drop(["Count"],axis=1)
# y=data["Count"]

In [103]:
monthly_split_index = monthly_processed_data[(monthly_processed_data['Year'] == 2018) & (monthly_processed_data['Month'] == 10)].iloc[-1].name
weekly_split_index = weekly_processed_data[(weekly_processed_data['Year'] == 2018) & (weekly_processed_data['Month'] == 11)].iloc[-1].name
print(monthly_split_index)

24662


In [104]:
# X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.2,random_state=42)
def get_X_y(df,split_index):
    X_train = df.iloc[:split_index].drop('Count', axis=1)
    X_test = df.iloc[split_index:].drop('Count', axis=1)

    y_train = df.iloc[:split_index]['Count']
    y_test = df.iloc[split_index:]['Count']

    return X_train, X_test, y_train, y_test


In [105]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [129]:
def element_mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return (np.abs((y_true - y_pred) / y_true)) * 100

In [106]:
# weekly_processed_data[(weekly_processed_data['Year'] == 2018) & (weekly_processed_data['Month'] == 12) & (weekly_processed_data['wom'] == 1) & (weekly_processed_data['wom'] == 2)]

In [122]:
def weekly_prediction_cases(regressor):
     week_ahead = weekly_processed_data[(weekly_processed_data['Year'] == 2018) & (weekly_processed_data['Month'] == 12) & (weekly_processed_data['wom'] == 1)]
     
     two_weeks_ahead = weekly_processed_data[(weekly_processed_data['Year'] == 2018) & (weekly_processed_data['Month'] == 12) & ((weekly_processed_data['wom'] == 1) | (weekly_processed_data['wom'] == 2))]
     

     week_ahead_pred = regressor.predict(week_ahead.drop('Count', axis=1))
     two_weeks_ahead_pred = regressor.predict(two_weeks_ahead.drop('Count', axis=1))
     # half_year_pred = regressor.predict(half_year.drop('Count', axis=1))
     
     err = mean_absolute_error(week_ahead['Count'], week_ahead_pred)
     week_ahead_MAPE = mean_absolute_percentage_error(week_ahead['Count'], week_ahead_pred)
     two_weeks_ahead_MAPE = mean_absolute_percentage_error( two_weeks_ahead['Count'], two_weeks_ahead_pred)
     # half_year_MAPE = mean_absolute_percentage_error(half_year['Count'], half_year_pred) 
     
     week_ahead["predictions"]=week_ahead_pred
     two_weeks_ahead["predictions"]=two_weeks_ahead_pred
     # half_year["predictions"]=half_year_pred
     week_ahead=decode_weekly_names(week_ahead)
     two_weeks_ahead=decode_weekly_names(two_weeks_ahead)


     week_ahead=week_ahead.merge(prices,on=["Name"])
     week_ahead["Actual_month_Cost"]=week_ahead.Count * week_ahead["Avg Unit Price"]
     week_ahead["Prediction_month_Cost"]=week_ahead.predictions * week_ahead["Avg Unit Price"]


     week_ahead["Actual_month_Cost"]=week_ahead.Count * week_ahead["Avg Unit Price"]
     week_ahead["Prediction_month_Cost"]=week_ahead.predictions * week_ahead["Avg Unit Price"]     


     # half_year['Service Name']=half_year['Service Name'].map(Service_Index_to_name)

     # week_ahead['Service Name AR']=week_ahead['Service Name'].map(operations_names_en_to_ar)
     # two_weeks_ahead['Service Name AR']=two_weeks_ahead['Service Name'].map(operations_names_en_to_ar)
     # half_year['Service Name AR']=half_year['Service Name'].map(operations_names_en_to_ar)     

     return err,week_ahead_MAPE, two_weeks_ahead_MAPE, week_ahead, two_weeks_ahead

In [123]:
def monthly_prediction_cases(regressor):
     month_ahead = monthly_processed_data[(monthly_processed_data['Year'] == 2018) & (monthly_processed_data['Month'] == 12)]
     #.sort_values(by=["Count"],ascending=False).iloc[0:10]
     
     quarter_year = monthly_processed_data[(monthly_processed_data['Year'] == 2018) & (monthly_processed_data['Month'] >= 9)]
     
     half_year = monthly_processed_data[(monthly_processed_data['Year'] == 2018) & (monthly_processed_data['Month'] >= 7)]

     month_ahead_pred = regressor.predict(month_ahead.drop('Count', axis=1))
     quarter_year_pred = regressor.predict(quarter_year.drop('Count', axis=1))
     half_year_pred = regressor.predict(half_year.drop('Count', axis=1))
     
     err = mean_absolute_error(month_ahead['Count'], month_ahead_pred)
     month_MAPE = mean_absolute_percentage_error(month_ahead['Count'], month_ahead_pred)
     quarter_year_MAPE = mean_absolute_percentage_error( quarter_year['Count'], quarter_year_pred)
     half_year_MAPE = mean_absolute_percentage_error(half_year['Count'], half_year_pred) 
     
     month_ahead["predictions"]=month_ahead_pred
     quarter_year["predictions"]=quarter_year_pred
     half_year["predictions"]=half_year_pred

     month_ahead=decode_monthly_names(month_ahead)
     quarter_year=decode_monthly_names(quarter_year)
     half_year=decode_monthly_names(half_year)

     month_MAPE = mean_absolute_percentage_error(month_ahead['Count'], month_ahead_pred)
     element_month_MAPE = element_mean_absolute_percentage_error(month_ahead['Count'], month_ahead_pred)
     month_ahead["Month_MAPE"]=element_month_MAPE

     month_ahead=month_ahead.merge(prices,on=["Name"])
     month_ahead["Actual_month_Cost"]=month_ahead.Count * month_ahead["Avg Unit Price"]
     month_ahead["Prediction_month_Cost"]=month_ahead.predictions * month_ahead["Avg Unit Price"]

     month_ahead=month_ahead.merge(inv_df,left_on=["Name","Month","Year"],right_on=["Type","Month","Year"])
     month_ahead["Inventory_month_Cost"]=month_ahead.Inventory_count * month_ahead.Inventory_price


     month_ahead["subtract_inventory_prediction"]=  month_ahead["Inventory_month_Cost"] -month_ahead["Prediction_month_Cost"]
     month_ahead["subtract_inventory_actual"]=  month_ahead["Inventory_month_Cost"]- month_ahead["Actual_month_Cost"]


     # month_ahead['Service Name']=month_ahead['Service Name'].map(Service_Index_to_name)
     # quarter_year['Service Name']=quarter_year['Service Name'].map(Service_Index_to_name)
     # half_year['Service Name']=half_year['Service Name'].map(Service_Index_to_name)
     

     # month_ahead['Service Name AR']=month_ahead['Service Name'].map(operations_names_en_to_ar)
     # quarter_year['Service Name AR']=quarter_year['Service Name'].map(operations_names_en_to_ar)
     # half_year['Service Name AR']=half_year['Service Name'].map(operations_names_en_to_ar)     
     # month_ahead['MAPE']=


     return err,month_MAPE,quarter_year_MAPE, half_year_MAPE, month_ahead,quarter_year, half_year

In [124]:
def perform_weekly_regression(regressor):
    # regressor = DecisionTreeRegressor(random_state=0)
    X_train, X_test, y_train, y_test=get_X_y(weekly_processed_data,weekly_split_index)
    regressor.fit(X_train, y_train)
    # y_pred = regressor.predict(X_test)
    # print(f'Total MAE {mean_absolute_percentage_error(y_pred, y_test)}')
    err,week_ahead_MAPE, two_weeks_ahead_MAPE, week_ahead, two_weeks_ahead= weekly_prediction_cases(regressor)

    print(f'1 week predection MAE {err}')
    print(f'1 week predection MAPE {week_ahead_MAPE}')
    print(f'2 weeks predection MAPE {two_weeks_ahead_MAPE}')
    # print(f'half year MAPE {half_year_MAPE}')
    return week_ahead, two_weeks_ahead

In [125]:
def perform_monthly_regression(regressor):
    # regressor = DecisionTreeRegressor(random_state=0)
    X_train, X_test, y_train, y_test=get_X_y(monthly_processed_data,monthly_split_index)
    regressor.fit(X_train, y_train)
    # y_pred = regressor.predict(X_test)
    # print(f'Total MAE {mean_absolute_percentage_error(y_pred, y_test)}')
    err,month_MAPE,quarter_year_MAPE, half_year_MAPE, month_ahead,quarter_year, half_year = monthly_prediction_cases(regressor)

    print(f'1 month predection MAE {err}')
    print(f'1 month predection MAPE {month_MAPE}')
    print(f'Last quarter year MAPE {quarter_year_MAPE}')
    print(f'half year MAPE {half_year_MAPE}')
    return month_ahead,quarter_year, half_year

In [111]:
def vis(df):
    fig = go.Figure(data=[
                        go.Bar(name = 'Real Data', x=df["Name"], y=df["Count"], marker_color='#800000'),
                        go.Bar(name = 'Predicted', x=df["Name"], y=df["predictions"], marker_color='#228B22'),

    ])

    fig.update_layout(barmode='group',width=1200, height=400)
    return fig.show()

1. **Decision Tree Regression**

In [112]:
regressor = DecisionTreeRegressor(random_state=0)
tw=perform_weekly_regression(regressor)

1 week predection MAE 67.01388888888889
1 week predection MAPE 41.572943485682536
2 weeks predection MAPE 38.95491496693732


In [25]:
vis(tw[0])

In [26]:
tw[0].head()

,Name,Count,wom,Month,Year,predictions,Avg Unit Price,Actual_month_Cost,Prediction_month_Cost
0,\tبانش 7.75\t,17,1,12,2018,3.0,1066.666667,18133.333333,3200.000000
1,\tبانش 8\t,16,1,12,2018,4.0,1066.666667,17066.666667,4266.666667
2,\tبانش 8.25\t,14,1,12,2018,3.0,1066.666667,14933.333333,3200.000000
3,\tبانش 8.5\t,16,1,12,2018,4.0,1066.666667,17066.666667,4266.666667
4,\tبروب أمامى Centurion\t,264,1,12,2018,188.0,1239.222222,327154.666667,232973.777778


In [126]:
regressor = DecisionTreeRegressor(random_state=0)
tm=perform_monthly_regression(regressor)

1 month predection MAE 20.15745856353591
1 month predection MAPE 9.335876733048204
Last quarter year MAPE 5.586039107542428
half year MAPE 3.7240260716949525


In [114]:
vis(tm[0])

** Analysis on the prediction and actual data prices: **

In [127]:
pred_df=tm[0]

# pred_df.to_csv("DATA.csv",encoding='utf-8-sig')

In [128]:
pred_df.head()

,Name,Count,Month,Year,predictions,Month_MAPE,Avg Unit Price,Actual_month_Cost,Prediction_month_Cost,Type,Inventory_count,Inventory_price,Inventory_month_Cost,subtract_inventory_prediction,subtract_inventory_actual
0,أوبسيت صغير,1809,12,2018,1844.0,1.934771,9.333333,16884.000000,17210.666667,أوبسيت صغير,600,9.333333,5.600000e+03,-1.161067e+04,-1.128400e+04
1,أيباك صغير,2096,12,2018,2074.0,1.049618,8.285263,17365.911579,17183.635789,أيباك صغير,8100,8.285263,6.711063e+04,4.992700e+04,4.974472e+04
2,أيباك كبير,1874,12,2018,1900.0,1.387407,44.331111,83076.502222,84229.111111,أيباك كبير,587,44.331111,2.602236e+04,-5.820675e+04,-5.705414e+04
3,بليد Femto Kit,427,12,2018,474.0,11.007026,998.232500,426245.277500,473162.205000,بليد Femto Kit,7840,998.232500,7.826143e+06,7.352981e+06,7.399898e+06
4,جاون مريض (XL),489,12,2018,544.0,11.247444,4.978161,2434.320690,2708.119540,جاون مريض (XL),4310,4.978161,2.145587e+04,1.874775e+04,1.902155e+04


In [52]:
print(f"The Actual total cost of the SKUs of one month is {pred_df.Actual_month_Cost.sum()} ")
print(f"The Prediction total cost of the SKUs of one month is {pred_df.Prediction_month_Cost.sum()} ")
print(f"The Inventory total cost of the SKUs of one month is {pred_df.Inventory_month_Cost.sum()} ")

The Actual total cost of the SKUs of one month is 1792905.1380431124 
The Prediction total cost of the SKUs of one month is 1862502.4535453895 
The Inventory total cost of the SKUs of one month is 9184466.601220598 


** Saved Capital From the Prediction Values**

In [53]:
print(f"The proposed capital to be saved from the inventory spending is   {pred_df.Inventory_month_Cost.sum()-pred_df.Prediction_month_Cost.sum()} ")

The proposed capital to be saved from the inventory spending is   7321964.147675209 


** Saved Capital From the Actual Values**

In [54]:
print(f"The proposed capital to be saved from the inventory spending is   {pred_df.Inventory_month_Cost.sum()-pred_df.Actual_month_Cost.sum()} ")

The proposed capital to be saved from the inventory spending is   7391561.463177486 


2. **Support Vector Machine Regression**

In [38]:
# svr = SVR()
# perform_monthly_regression(svr)

3. **AdaBoost Regression**

In [39]:
# ada = AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='mae'), n_estimators=200, learning_rate=0.02, loss='square',random_state=72)
# x=perform_weekly_regression(ada)
# vis(x[0])

In [40]:
# ada = AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='mae'), n_estimators=300, learning_rate=0.01, loss='square',random_state=7)
# x=perform_monthly_regression(ada)
# vis(x[0]) 

In [41]:
# xgb_model = xgb.XGBRegressor(objective="reg:squarederror", random_state=7)
# perform_monthly_regression(xgb_model)

In [42]:
# gbc=GradientBoostingRegressor(n_estimators=500,max_depth=8,min_samples_split=2,loss="ls",random_state=7)
# perform_monthly_regression(gbc)

** choosing the best Model  till now which is (ADABOOSTING) so We will pickle it **

In [43]:
# with open('./D_Tree_regressor.pickle', 'wb') as f:
#     pickle.dump(regressor, f)

**Testing the pickled model**

In [44]:
with open('./D_Tree_regressor.pickle', 'rb') as f:
  reg = pickle.load(f)
month=perform_monthly_regression(reg)

1 month predection MAE 20.15745856353591
1 month predection MAPE 9.335876733048204
Last quarter year MAPE 5.586039107542428
half year MAPE 3.7240260716949525


In [45]:
week=perform_weekly_regression(reg)

1 week predection MAE 67.01388888888889
1 week predection MAPE 41.572943485682536
2 weeks predection MAPE 38.95491496693732


In [46]:
# week=perform_weekly_regression(reg)
# month[0].iloc[0:60]

In [47]:
week[0].to_csv("pred_SKU_one_week_D_Tree.csv",encoding='utf-8-sig')
month[0].to_csv("pred_SKU_one_month_D_Tree.csv",encoding='utf-8-sig')
# tm[0].to_csv("pred_one_month_decision_tree.csv",encoding='utf-8-sig')
# half_year.to_csv("pred_half_year_ada.csv",encoding='utf-8-sig')